In [7]:
import os
import json
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from src import settings
from src.utils import fileio, graph_utils


INPUT_DIR = os.path.join(
    settings.RESULTS_DIR, "distances_traveled", TREATMENT
)
distances = fileio.load_files_from_folder(INPUT_DIR, ".csv")

SCRIPT_OUTPUT = os.path.join(
    settings.OUTPUT_DIR, "1_1_create_snapshots", TREATMENT, "120_sec_window"
)
interactions = fileio.load_multiple_folders(INPUT_DIR)

group = []
for distances_name, distances_path in distances.items():
    interactions_path = interactions[distances_name.replace(".csv","")] 
    interactions_group = fileio.load_files_from_folder(interactions_path, ".gml")

    res = []
    for graph_name, graph_path in interactions_group.items():
        G = nx.read_gml(graph_path)
        out_deg = G.degree()
        res.append(dict(out_deg))

    summed_dict = {}

    for dictionary in res:
        # Iterate over the keys in the current dictionary
        for key in dictionary.keys():
            # Add the corresponding value to the sum, or initialize it if not present
            summed_dict[key] = summed_dict.get(key, 0) + dictionary[key]

    distances = pd.read_csv(distances_path,index_col=0)
    distances = distances.sum()

    for key in summed_dict.keys():
        group.append((summed_dict[key], distances[key], distances_name.replace(".csv", "")))


x = [item[0] for item in group]
y = [item[1] for item in group]
groups = [item[2] for item in group]
unique_groups = list(set(groups))
num_groups = len(unique_groups)

colormap = plt.cm.get_cmap('tab10', num_groups)
group_colors = [colormap(i) for i in range(num_groups)]
group_color_map = dict(zip(unique_groups, group_colors))
colors = [group_color_map[group] for group in groups]

# Plot scatter plot
plt.scatter(x, y, c=colors)

legend_elements = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, label=group) for group, color in group_color_map.items()]
plt.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))

plt.ylabel('Distance')
plt.xlabel('Number of Interactions')
plt.title('Scatter Plot: Distance vs Number of Interactions')
plt.show()

SystemExit: Directory is empty!

/home/milky/Envs/drosophilaSNA/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
distances

{}

In [6]:
distances

{}

In [6]:
group

[(45, 465505.24435399513, 'CsCh_A4_21_04_2023-11_54'),
 (38, 334377.04368304595, 'CsCh_A4_21_04_2023-11_54'),
 (45, 348622.76687342767, 'CsCh_A4_21_04_2023-11_54'),
 (59, 455736.84711256967, 'CsCh_A4_21_04_2023-11_54'),
 (47, 373101.98736320157, 'CsCh_A4_21_04_2023-11_54'),
 (60, 353422.4519738189, 'CsCh_A4_21_04_2023-11_54'),
 (36, 460673.5154279832, 'CsCh_A4_21_04_2023-11_54'),
 (29, 504132.7534776922, 'CsCh_A4_21_04_2023-11_54'),
 (45, 400891.44564050727, 'CsCh_A4_21_04_2023-11_54'),
 (46, 391892.4278030797, 'CsCh_A4_21_04_2023-11_54'),
 (46, 371513.2712163707, 'CsCh_A4_21_04_2023-11_54'),
 (53, 392048.625522207, 'CsCh_A4_21_04_2023-11_54'),
 (53, 408035.695748217, 'CsCh_A1_05_05_2023-11_21'),
 (52, 302279.8824971552, 'CsCh_A1_05_05_2023-11_21'),
 (48, 328958.0748054405, 'CsCh_A1_05_05_2023-11_21'),
 (53, 251241.66544170867, 'CsCh_A1_05_05_2023-11_21'),
 (58, 315873.06345705956, 'CsCh_A1_05_05_2023-11_21'),
 (46, 298326.654297822, 'CsCh_A1_05_05_2023-11_21'),
 (60, 297997.6171844413